# KarpovCourses. Продуктовая аналитика 
## Сегментация клиентов 


### Домашнее задание. RFM-анализ

In [ ]:
import pandas as pd
import seaborn as sns
sns.set_theme(
    style="whitegrid",
    rc={"figure.figsize": (7, 3.5)})

In [2]:
orders = pd.read_csv('4_RFM_ht_data.csv', parse_dates=['InvoiceDate'], low_memory=False)

In [10]:
orders.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [8]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     332730 non-null  object        
 1   CustomerCode  332730 non-null  object        
 2   InvoiceDate   332730 non-null  datetime64[ns]
 3   Amount        332730 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 10.2+ MB


In [9]:
orders.describe()

,InvoiceDate,Amount
count,332730,332730.000000
mean,2020-09-15 16:35:03.846361856,1239.029737
min,2020-09-01 00:00:00,-8925.000000
25%,2020-09-08 00:00:00,350.000000
50%,2020-09-16 00:00:00,720.445000
75%,2020-09-24 00:00:00,1491.507500
max,2020-09-30 00:00:00,131874.000000
std,NaN,1747.362217


In [6]:
orders['InvoiceDate'] = pd.to_datetime(orders['InvoiceDate'])
orders['CustomerCode'] = orders['CustomerCode'].apply(str)
orders['InvoiceNo'] = orders['InvoiceNo'].apply(str)

In [12]:
orders['CustomerCode'].nunique() 

123733

### RFM-анализ

In [13]:
last_date = orders['InvoiceDate'].max()

In [ ]:
# вариант из ответов - мне кажется громоздким, поэтому заменил своим вариантом
rfmTable = orders.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'InvoiceNo': lambda x: len(x),      # Frequency #Количество заказов
                                        'Amount': lambda x: x.sum()}) # Monetary Value #Общая сумма по всем заказам
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)

In [27]:
rfmTable = orders.groupby('CustomerCode', as_index=False).agg(recency=('InvoiceDate', lambda x: (last_date - x.max()).days), frequency=('InvoiceNo', 'count'), monetary_value=('Amount','sum'))
rfmTable.head()

,CustomerCode,recency,frequency,monetary_value
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88


In [40]:
rfm = rfmTable

In [ ]:
# код из ответов - не правильный (пытается применить квантили ко всем колонкам, в том числе к колонке CustomerCode, которая может содержать строковые значения)
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [31]:
quantiles = rfmTable[['recency', 'frequency', 'monetary_value']].quantile([0.25, 0.5, 0.75])

In [32]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [33]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4

def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1

In [41]:
rfm['R_Quartile'] = rfm['recency'].apply(RClass, args=('recency',quantiles))
rfm['F_Quartile'] = rfm['frequency'].apply(FMClass, args=('frequency',quantiles))
rfm['M_Quartile'] = rfm['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))
rfm['RFMClass'] = rfm.R_Quartile.map(str) + rfm.F_Quartile.map(str) + rfm.M_Quartile.map(str)

In [42]:
rfm.head()

,CustomerCode,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443


In [43]:
# проверка - должно совпадать с уникальными строками по полю 'CustomerCode' в оригинальной таблице
rfm.shape[0]

123733

Какое максимальное кол-во покупок было совершено одним пользователем?

In [44]:
rfm['frequency'].max()

np.int64(204)

Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)

In [45]:
rfm[rfm['M_Quartile'] == 4].monetary_value.max()

np.float64(765.0)

Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?

In [49]:
rfm[rfm['F_Quartile'] == 1].frequency.min()

np.int64(4)

Какая верхняя граница у количества покупок у пользователей с классом 2 в подсегменте R?

In [50]:
rfm[rfm['R_Quartile'] == 2].frequency.max()

np.int64(49)

Сколько пользователей попало в сегмент 111?

In [52]:
rfm[rfm['RFMClass'] == '111'].shape[0]

9705

Сколько пользователей попало в сегмент 311?

In [53]:
rfm[rfm['RFMClass'] == '311'].shape[0]

1609

В каком RFM-сегменте самое большое кол-во пользователей?

In [54]:
rfm.groupby('RFMClass').size().sort_values(ascending = False)

RFMClass
444    10624
111     9705
443     6729
344     6593
211     5847
       ...  
424       63
214       60
114       60
314       33
414        2
Length: 64, dtype: int64